In [1]:
import json

#Validation Set
with open('val.json') as json_file:
    val = json.load(json_file)
episodes = val['episodes']
type(episodes)

list

In [2]:
question_vocab_val = val['question_vocab']
answer_vocab_val = val['answer_vocab']

In [3]:
# Check for Unknowns
qs = [item['question'] for item in episodes]
qt = [item['question_text'] for item in qs]

for q in qt:
    if '<unk>' in q:
        print(q)
        
ans = [item['answer_text'] for item in qs]
for a in ans:
    if '<unk>' in a:
        print(a)

In [4]:
#Unique Answers
unique_as = set(a for a in ans)
unique_as

{'bedroom',
 'black',
 'blue',
 'brown',
 'closet',
 'family room',
 'green',
 'grey',
 'gym',
 'hallway',
 'kitchen',
 'laundry room',
 'living room',
 'lounge',
 'off-white',
 'olive green',
 'red',
 'red brown',
 'silver',
 'slate grey',
 'spa',
 'tan',
 'white',
 'yellow'}

In [5]:
#Train Set
with open('train.json') as json_file:
    train = json.load(json_file)
train_episodes = train['episodes']
type(train_episodes)

list

In [6]:
answer_vocab_train = train['answer_vocab']
question_vocab_train = train['question_vocab']

In [7]:
# Check for Unknowns
tqs = [item['question'] for item in train_episodes]
tqt = [item['question_text'] for item in tqs]

for tq in tqt:
    if '<unk>' in tq:
        print(tq)
        
tans = [item['answer_text'] for item in tqs]
for ta in tans:
    if '<unk>' in ta:
        print(ta)

In [8]:
#Unique Answers
unique_tas = set(ta for ta in tans)
unique_tas

{'bathroom',
 'bedroom',
 'black',
 'blue',
 'brown',
 'closet',
 'dining room',
 'family room',
 'foyer',
 'green',
 'grey',
 'hallway',
 'kitchen',
 'laundry room',
 'light blue',
 'living room',
 'lounge',
 'off-white',
 'office',
 'olive green',
 'orange yellow',
 'purple',
 'purple pink',
 'red',
 'red brown',
 'silver',
 'slate grey',
 'spa',
 'tan',
 'tv room',
 'white',
 'yellow',
 'yellow green',
 'yellow pink'}

In [9]:
val_only = set(unique_as - unique_tas)
val_only

{'gym'}

In [10]:
#Making a mini-version of the dataset for local testing
episodes[0]

{'episode_id': '1',
 'scene_id': 'mp3d/2azQ1b91cZZ/2azQ1b91cZZ.glb',
 'start_position': [2.4317048470246725, 3.9743599128723126, 7.183146352910121],
 'start_rotation': [-7.075170913732151e-16,
  0.9969173337297776,
  0.0,
  0.07845909577041682],
 'info': {'bboxes': [{'type': 'object',
    'box': {'centroid': [0.735851, 1.50759, 5.46594],
     'a0': [-1.0, 2.22045e-16, 0.0],
     'a1': [-2.22045e-16, -1.0, 0.0],
     'a2': [0.0, 0.0, 1.0],
     'radii': [0.308081, 0.0541849, 0.310809],
     'obj_id': 827,
     'level': 1,
     'room_id': 26},
    'name': 'picture',
    'target': True},
   {'type': 'room',
    'box': {'centroid': [1.022561, 0.05626500000000001, 4.82052],
     'a0': [1.0, 0.0, 0.0],
     'a1': [0.0, 1.0, 0.0],
     'a2': [0.0, 0.0, 1.0],
     'radii': [1.7039689999999998, 1.505505, 1.41371],
     'room_id': 26,
     'level': 1},
    'name': ['closet'],
    'target': False}],
  'question_meta': [{'name': 'color', 'diffuse': 'brown'}],
  'question_answers_entropy': 0.916269

In [11]:
# matterport scenes to use in mini-dataset
scenes = ['mp3d/1LXtFkjw3qL/1LXtFkjw3qL.glb', 'mp3d/1pXnuDYAj8r/1pXnuDYAj8r.glb', 'mp3d/2azQ1b91cZZ/2azQ1b91cZZ.glb', 'mp3d/2n8kARJN3HM/2n8kARJN3HM.glb']

In [12]:
mini_val_episodes = list(filter(lambda d: d['scene_id'] in scenes, episodes))
len(mini_val_episodes)

225

In [13]:
mini_val = {'episodes': mini_val_episodes, 'question_vocab': question_vocab_val, 'answer_vocab': answer_vocab_val}

In [14]:
mini_train_episodes = list(filter(lambda d: d['scene_id'] in scenes, train_episodes))
mini_train = {'episodes': mini_train_episodes, 'question_vocab': question_vocab_train, 'answer_vocab': answer_vocab_train}

In [15]:
len(mini_train_episodes)

795

In [16]:
#write to json files

with open('mini_train.json', 'w') as f:
    json.dump(mini_train, f)
    
with open('mini_val.json', 'w') as f:
    json.dump(mini_val, f)